In [1]:
import numpy as np
import itertools
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime, timedelta
from scipy import sparse
%matplotlib inline

sns.set_theme(style="ticks")

In [2]:
df_all = pd.read_csv('../data/Booking/booking_train_set.csv', 
                 dtype={"user_id": str, "city_id": str, 'affiliate_id': str,
                       'utrip_id': str},parse_dates=['checkin', 'checkout'])

df_all.head()

,Unnamed: 0,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id
0,0,1006220,2016-04-09,2016-04-11,31114,desktop,384,Gondal,Gondal,1006220_1
1,1,1006220,2016-04-11,2016-04-12,39641,desktop,384,Gondal,Gondal,1006220_1
2,2,1006220,2016-04-12,2016-04-16,20232,desktop,384,Gondal,Glubbdubdrib,1006220_1
3,3,1006220,2016-04-16,2016-04-17,24144,desktop,384,Gondal,Gondal,1006220_1
4,4,1010293,2016-07-09,2016-07-10,5325,mobile,359,The Devilfire Empire,Cobra Island,1010293_1


In [3]:
df = pd.read_csv('../output/booking/dataset/train_500_30_10.csv')

df.head()

,utrip_id,user_id,user_features,count_unique_city,trip_size,start_trip,end_trip,checkin_list,checkout_list,days_since_2016_list,...,affiliate_id_list,booker_country_list,hotel_country_list,step_list,first_city_id,first_hotel_country,last_city_id,last_hotel_country,country_count,duration_sum
0,1000027_1,1000027,"[-0.2721548080444336, -0.3261375427246094, -0....",4,3,2016-08-13,2016-08-18,"['0', '0', '0', '0', '0', '0', '0', '2016-08-1...","['0', '0', '0', '0', '0', '0', '0', '2016-08-1...","[0, 0, 0, 0, 0, 0, 0, 225, 226, 228]",...,"['0', '0', '0', '0', '0', '0', '0', '7168', '7...","['0', '0', '0', '0', '0', '0', '0', 'Elbonia',...","['0', '0', '0', '0', '0', '0', '0', 'Gondal', ...","[0, 0, 0, 0, 0, 0, 0, 1, 2, 3]",8183,Gondal,30628,Gondal,88076,5
1,1000033_1,1000033,"[-0.653695285320282, 0.9078158736228943, 0.579...",4,4,2016-04-09,2016-04-16,"['0', '0', '0', '0', '0', '0', '2016-04-09', '...","['0', '0', '0', '0', '0', '0', '2016-04-11', '...","[0, 0, 0, 0, 0, 0, 99, 101, 102, 104]",...,"['0', '0', '0', '0', '0', '0', '359', '384', '...","['0', '0', '0', '0', '0', '0', 'Gondal', 'Gond...","['0', '0', '0', '0', '0', '0', 'Cobra Island',...","[0, 0, 0, 0, 0, 0, 1, 2, 3, 4]",38677,Cobra Island,38677,Cobra Island,96654,7
2,1000045_1,1000045,"[-1.1103595495224, -1.2900782823562622, -0.307...",7,6,2016-06-18,2016-06-28,"['0', '0', '0', '0', '2016-06-18', '2016-06-20...","['0', '0', '0', '0', '2016-06-20', '2016-06-22...","[0, 0, 0, 0, 169, 171, 173, 175, 176, 177]",...,"['0', '0', '0', '0', '2790', '7974', '7974', '...","['0', '0', '0', '0', 'The Devilfire Empire', '...","['0', '0', '0', '0', 'Fook Island', 'Fook Isla...","[0, 0, 0, 0, 1, 2, 3, 4, 5, 6]",64876,Fook Island,36063,Gondal,16624,10
3,1000083_1,1000083,"[1.3209058046340942, 0.19926407933235168, 0.57...",4,3,2016-06-13,2016-06-16,"['0', '0', '0', '0', '0', '0', '0', '2016-06-1...","['0', '0', '0', '0', '0', '0', '0', '2016-06-1...","[0, 0, 0, 0, 0, 0, 0, 164, 165, 166]",...,"['0', '0', '0', '0', '0', '0', '0', '359', '35...","['0', '0', '0', '0', '0', '0', '0', 'The Devil...","['0', '0', '0', '0', '0', '0', '0', 'Osterlich...","[0, 0, 0, 0, 0, 0, 0, 1, 2, 3]",55990,Osterlich,36063,Gondal,13913,3
4,100008_1,100008,"[-0.346758633852005, 0.11678697913885117, -1.0...",5,4,2016-07-18,2016-07-25,"['0', '0', '0', '0', '0', '0', '2016-07-18', '...","['0', '0', '0', '0', '0', '0', '2016-07-21', '...","[0, 0, 0, 0, 0, 0, 199, 202, 203, 204]",...,"['0', '0', '0', '0', '0', '0', '8436', '8436',...","['0', '0', '0', '0', '0', '0', 'Gondal', 'Gond...","['0', '0', '0', '0', '0', '0', 'Kamistan', 'Ka...","[0, 0, 0, 0, 0, 0, 1, 2, 3, 4]",11306,Kamistan,65690,Kamistan,6480,7


In [4]:
df['city_id_list'] = df['city_id_list'].apply(eval)
df['city_id_list']

0                 [0, 0, 0, 0, 0, 0, 0, 8183, 15626, 60902]
1            [0, 0, 0, 0, 0, 0, 38677, 52089, 21328, 27485]
2         [0, 0, 0, 0, 64876, 55128, 9608, 31817, 36170,...
3                [0, 0, 0, 0, 0, 0, 0, 55990, 14705, 35160]
4              [0, 0, 0, 0, 0, 0, 11306, 12096, 6761, 6779]
                                ...                        
912359               [0, 0, 0, 0, 0, 0, 0, 0, 51291, 66969]
912360           [0, 0, 0, 0, 0, 0, 0, 51291, 66969, 67169]
912361                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 17944]
912362               [0, 0, 0, 0, 0, 0, 0, 0, 17944, 47075]
912363             [0, 0, 0, 0, 0, 0, 0, 17944, 47075, 228]
Name: city_id_list, Length: 912364, dtype: object

## Coocorrence

In [5]:
def create_co_occurences_matrix(allowed_words, documents):
    word_to_id       = dict(zip(allowed_words, range(len(allowed_words))))
    documents_as_ids = [np.sort([word_to_id[w] for w in doc if w in word_to_id]).astype('uint32') for doc in documents]
    row_ind, col_ind = zip(*itertools.chain(*[[(i, w) for w in doc] for i, doc in enumerate(documents_as_ids)]))
    
    data        = np.ones(len(row_ind), dtype='uint32')  # use unsigned int for better memory utilization
    max_word_id = max(itertools.chain(*documents_as_ids)) + 1
    
    docs_words_matrix = csr_matrix((data, (row_ind, col_ind)), shape=(len(documents_as_ids), max_word_id))  # efficient arithmetic operations with CSR * CSR
    words_cooc_matrix = docs_words_matrix.T * docs_words_matrix  # multiplying docs_words_matrix with its transpose matrix would generate the co-occurences matrix
    words_cooc_matrix.setdiag(0)

    return words_cooc_matrix, word_to_id 

In [6]:
item_idx = np.unique(df_all['city_id'])
lists    = list(df['city_id_list'])

In [7]:
cooc_matrix, cooc_to_idx = create_co_occurences_matrix(item_idx, lists)
cooc_to_id =  {v: k for k, v in cooc_to_idx.items()}

/home/marlesson/anaconda3/envs/booking_challenge/lib/python3.6/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [8]:
uid    = cooc_to_idx['29770'] #52818
scores = np.argsort(np.array(cooc_matrix[uid].todense())[0])[::-1]
scores, np.max(scores)

(array([29690,  7751,  4223, ..., 26375, 26374,     0]), 39900)

In [9]:
[(cooc_to_id[_id], cooc_matrix[uid,_id]) for _id in scores[:10]]

[('55196', 1904),
 ('21929', 1525),
 ('16521', 1093),
 ('48343', 1074),
 ('46258', 1056),
 ('52818', 958),
 ('44869', 750),
 ('37601', 707),
 ('64269', 677),
 ('36063', 626)]

In [10]:
cooc_matrix.shape

(39901, 39901)

In [11]:
def get_neighbors(uid,cooc_matrix, cooc_to_idx,cooc_to_id, top=10):
    if uid in cooc_to_idx:
        uid    = cooc_to_idx[uid] 
        scores = np.argsort(np.array(cooc_matrix[uid].todense())[0])[::-1]

        return [cooc_to_id[_id] for _id in scores[:top]],\
                    [cooc_matrix[uid,_id] for _id in scores[:top]]
    else:
        return [], []

In [12]:
uid = '29770'
items, scores = get_neighbors(uid, cooc_matrix, cooc_to_idx, cooc_to_id, top=10)
list(zip(items, scores))

[('55196', 1904),
 ('21929', 1525),
 ('16521', 1093),
 ('48343', 1074),
 ('46258', 1056),
 ('52818', 958),
 ('44869', 750),
 ('37601', 707),
 ('64269', 677),
 ('36063', 626)]

## Session Similarity

In [13]:
sample_days = 500
test_days = 30
# Split Data
max_timestamp        = df_all.checkout.max()
init_train_timestamp = max_timestamp - timedelta(days = sample_days)
init_test_timestamp  = max_timestamp - timedelta(days = test_days)

# TODO Garantir que o usuário fique com a sessão no train ou test
df_train = df_all[(df_all.checkout >= init_train_timestamp) & (df_all.checkout < init_test_timestamp)]

df_train['visit'] = 1
df_train.head()

/home/marlesson/anaconda3/envs/booking_challenge/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,Unnamed: 0,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id,visit
0,0,1006220,2016-04-09,2016-04-11,31114,desktop,384,Gondal,Gondal,1006220_1,1
1,1,1006220,2016-04-11,2016-04-12,39641,desktop,384,Gondal,Gondal,1006220_1,1
2,2,1006220,2016-04-12,2016-04-16,20232,desktop,384,Gondal,Glubbdubdrib,1006220_1,1
3,3,1006220,2016-04-16,2016-04-17,24144,desktop,384,Gondal,Gondal,1006220_1,1
4,4,1010293,2016-07-09,2016-07-10,5325,mobile,359,The Devilfire Empire,Cobra Island,1010293_1,1


In [14]:
from pandas.api.types import CategoricalDtype

def create_sparse_matrix(df: pd.DataFrame):

    session_c = CategoricalDtype(sorted(df.utrip_id.unique()), ordered=True)
    item_c    = CategoricalDtype(sorted(df.city_id.unique()), ordered=True)

    col = df.utrip_id.astype(session_c).cat.codes
    row = df.city_id.astype(item_c).cat.codes
    sparse_matrix = csr_matrix((df["visit"], (row, col)), \
                            shape=(item_c.categories.size, session_c.categories.size))
    return sparse_matrix


In [15]:
item_idx = np.unique(df_train['city_id'])
lists    = list(df['city_id_list'])
sparse_matrix = create_sparse_matrix(df_train)
sparse_matrix

<39787x214146 sparse matrix of type '<class 'numpy.int64'>'
	with 1012282 stored elements in Compressed Sparse Row format>

In [16]:
matrix_item_idx  = dict(zip(item_idx, list(range(len(item_idx)))))
matrix_item_id   =  {v: k for k, v in matrix_item_idx.items()}
sparse_matrix    = sparse_matrix
cos_matrix       = cosine_similarity(sparse_matrix)
np.fill_diagonal(cos_matrix, 0)
cos_matrix       = sparse.csr_matrix(cos_matrix)

In [17]:
cos_matrix

<39787x39787 sparse matrix of type '<class 'numpy.float64'>'
	with 1408708 stored elements in Compressed Sparse Row format>

In [18]:
cos_matrix.shape

(39787, 39787)

In [19]:
# def get_neighbors(uid,cooc_matrix, cooc_to_idx,cooc_to_id, top=10):
#     uid    = cooc_to_idx[uid] 
#     scores = np.argsort(np.array(cooc_matrix[uid].todense())[0])[::-1]
    
#     return [(cooc_to_id[_id], cooc_matrix[uid,_id]) for _id in scores[:top]]

uid = '29770'
items, scores = get_neighbors(uid, cos_matrix, matrix_item_idx, matrix_item_id, top=10)
list(zip(items, scores))

[('55196', 0.2532611093140279),
 ('48343', 0.2095010966509785),
 ('46258', 0.20714425388042296),
 ('37601', 0.1763536469542316),
 ('21929', 0.16746105065885727),
 ('52818', 0.16626128258249187),
 ('2201', 0.15931144572277947),
 ('53831', 0.14325091442726673),
 ('19333', 0.13975019099745023),
 ('18083', 0.12923443443450883)]

## Save

In [20]:
df_all.head()

,Unnamed: 0,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id
0,0,1006220,2016-04-09,2016-04-11,31114,desktop,384,Gondal,Gondal,1006220_1
1,1,1006220,2016-04-11,2016-04-12,39641,desktop,384,Gondal,Gondal,1006220_1
2,2,1006220,2016-04-12,2016-04-16,20232,desktop,384,Gondal,Glubbdubdrib,1006220_1
3,3,1006220,2016-04-16,2016-04-17,24144,desktop,384,Gondal,Gondal,1006220_1
4,4,1010293,2016-07-09,2016-07-10,5325,mobile,359,The Devilfire Empire,Cobra Island,1010293_1


In [21]:
top_k     = 500
cities_id = list(np.unique(df_all['city_id']))
dict_cooc_map  = {c: cities_id for c in cities_id}
dict_sim_map   = {c: cities_id for c in cities_id}


for uid in cities_id:
    items, scores = get_neighbors(uid, cooc_matrix, cooc_to_idx, cooc_to_id, top=top_k)
    if len(items) > 0:
        dict_cooc_map[uid] = items
    
    items, scores = get_neighbors(uid, cos_matrix, matrix_item_idx, matrix_item_id, top=top_k)
    if len(items) > 0:
        dict_sim_map[uid]  = items

In [22]:
import pickle 

# SAVE
with open("../output/booking/dataset/neighbors_dict_cooc_map.pkl", "wb") as pkl_handle:
    pickle.dump(dict_cooc_map, pkl_handle)

with open("../output/booking/dataset/neighbors_dict_sim_map.pkl", "wb") as pkl_handle:
    pickle.dump(dict_sim_map, pkl_handle)
    
# # LOAD
# with open("../output/booking/dataset/data.pkl", "rb") as pkl_handle:
# 	output = pickle.load(pkl_handle)